In [459]:
import pandas as pd
import numpy as np
import tensorflow as tf
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

In [460]:
df = pd.read_csv('data/data_lagged.csv', index_col=0)
df

,CO(mg/m^3),Delta CO(mg/m^3),People,Suction Pump,Temperature(°C),Relative Humidity(%),Absolute Humidity(g/m^3),CO(mg/m^3)_prev_1,CO(mg/m^3)_prev_2,CO(mg/m^3)_prev_3,...,CO(mg/m^3)_lag_3,CO(mg/m^3)_lag_4,CO(mg/m^3)_lag_5,CO(mg/m^3)_lag_6,Delta CO(mg/m^3)_lag_1,Delta CO(mg/m^3)_lag_2,Delta CO(mg/m^3)_lag_3,Delta CO(mg/m^3)_lag_4,Delta CO(mg/m^3)_lag_5,Delta CO(mg/m^3)_lag_6
6,1.70,0.70,0.0,0.103795,9.712500,64.145833,0.770913,1.00,1.50,3.90,...,0.8,0.7,0.60,0.80,0.20,-0.5,-0.6,-0.1,-0.10,0.20
7,1.90,0.20,0.0,0.369293,9.587500,64.133333,0.765104,1.70,1.00,1.50,...,0.7,0.6,0.80,1.40,-0.50,-0.6,-0.1,-0.1,0.20,0.60
8,1.40,-0.50,0.0,0.408798,9.437500,64.220833,0.759387,1.90,1.70,1.00,...,0.6,0.8,1.40,4.40,-0.60,-0.1,-0.1,0.2,0.60,3.00
9,0.80,-0.60,0.0,0.243703,9.300000,64.308333,0.754154,1.40,1.90,1.70,...,0.8,1.4,4.40,3.75,-0.10,-0.1,0.2,0.6,3.00,-0.65
10,0.70,-0.10,0.0,0.122082,9.133333,64.533333,0.749083,0.80,1.40,1.90,...,1.4,4.4,3.75,3.10,-0.10,0.2,0.6,3.0,-0.65,-0.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,0.45,0.05,0.0,0.122082,18.566667,35.904167,0.658025,0.40,0.50,0.60,...,4.0,5.0,3.90,3.10,0.05,0.6,2.9,1.0,-1.10,-0.80
9323,0.50,0.05,0.0,0.063255,18.600000,36.275000,0.664142,0.45,0.40,0.50,...,5.0,3.9,3.10,2.40,0.60,2.9,1.0,-1.1,-0.80,-0.70
9324,1.10,0.60,4.0,0.000000,18.616667,36.720833,0.670212,0.50,0.45,0.40,...,3.9,3.1,2.40,2.40,2.90,1.0,-1.1,-0.8,-0.70,0.00
9325,4.00,2.90,8.0,0.000000,18.650000,37.104167,0.676096,1.10,0.50,0.45,...,3.1,2.4,2.40,2.10,1.00,-1.1,-0.8,-0.7,0.00,-0.30


In [461]:
df_X = df[df.columns[:-12]]
df_X

,CO(mg/m^3),Delta CO(mg/m^3),People,Suction Pump,Temperature(°C),Relative Humidity(%),Absolute Humidity(g/m^3),CO(mg/m^3)_prev_1,CO(mg/m^3)_prev_2,CO(mg/m^3)_prev_3,...,Absolute Humidity(g/m^3)_lag_2,Absolute Humidity(g/m^3)_prev_2,Absolute Humidity(g/m^3)_lag_3,Absolute Humidity(g/m^3)_prev_3,Absolute Humidity(g/m^3)_lag_4,Absolute Humidity(g/m^3)_prev_4,Absolute Humidity(g/m^3)_lag_5,Absolute Humidity(g/m^3)_prev_5,Absolute Humidity(g/m^3)_lag_6,Absolute Humidity(g/m^3)_prev_6
6,1.70,0.70,0.0,0.103795,9.712500,64.145833,0.770913,1.00,1.50,3.90,...,0.759387,0.780729,0.754154,0.784825,0.749083,0.786692,0.744362,0.784062,0.739700,0.779629
7,1.90,0.20,0.0,0.369293,9.587500,64.133333,0.765104,1.70,1.00,1.50,...,0.754154,0.775767,0.749083,0.780729,0.744362,0.784825,0.739700,0.786692,0.735113,0.784062
8,1.40,-0.50,0.0,0.408798,9.437500,64.220833,0.759387,1.90,1.70,1.00,...,0.749083,0.770913,0.744362,0.775767,0.739700,0.780729,0.735113,0.784825,0.731325,0.786692
9,0.80,-0.60,0.0,0.243703,9.300000,64.308333,0.754154,1.40,1.90,1.70,...,0.744362,0.765104,0.739700,0.770913,0.735113,0.775767,0.731325,0.780729,0.726617,0.784825
10,0.70,-0.10,0.0,0.122082,9.133333,64.533333,0.749083,0.80,1.40,1.90,...,0.739700,0.759387,0.735113,0.765104,0.731325,0.770913,0.726617,0.775767,0.721483,0.780729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,0.45,0.05,0.0,0.122082,18.566667,35.904167,0.658025,0.40,0.50,0.60,...,0.670212,0.644088,0.676096,0.636796,0.681867,0.628125,0.687208,0.617583,0.694262,0.606408
9323,0.50,0.05,0.0,0.063255,18.600000,36.275000,0.664142,0.45,0.40,0.50,...,0.676096,0.651271,0.681867,0.644088,0.687208,0.636796,0.694262,0.628125,0.700287,0.617583
9324,1.10,0.60,4.0,0.000000,18.616667,36.720833,0.670212,0.50,0.45,0.40,...,0.681867,0.658025,0.687208,0.651271,0.694262,0.644088,0.700287,0.636796,0.703671,0.628125
9325,4.00,2.90,8.0,0.000000,18.650000,37.104167,0.676096,1.10,0.50,0.45,...,0.687208,0.664142,0.694262,0.658025,0.700287,0.651271,0.703671,0.644088,0.702992,0.636796


In [462]:
df_y = df[df.columns[-12:-6]]
df_y

,CO(mg/m^3)_lag_1,CO(mg/m^3)_lag_2,CO(mg/m^3)_lag_3,CO(mg/m^3)_lag_4,CO(mg/m^3)_lag_5,CO(mg/m^3)_lag_6
6,1.9,1.4,0.8,0.7,0.60,0.80
7,1.4,0.8,0.7,0.6,0.80,1.40
8,0.8,0.7,0.6,0.8,1.40,4.40
9,0.7,0.6,0.8,1.4,4.40,3.75
10,0.6,0.8,1.4,4.4,3.75,3.10
...,...,...,...,...,...,...
9322,0.5,1.1,4.0,5.0,3.90,3.10
9323,1.1,4.0,5.0,3.9,3.10,2.40
9324,4.0,5.0,3.9,3.1,2.40,2.40
9325,5.0,3.9,3.1,2.4,2.40,2.10


In [463]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, shuffle=True, random_state=42)

In [464]:
# create validation set
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

In [465]:
scaler = StandardScaler()# #MinMaxScaler() #RobustScaler(quantile_range=(5,95)) #RobustScaler(quantile_range=(1,99))#
X_train = scaler.fit_transform(X_train)
#X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [466]:
# create model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(300, input_dim=X_train.shape[1], activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dropout(0.05),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(y_train.shape[1], activation='relu')
])

model.compile(optimizer="Adam", loss='mae')

In [467]:
# train model
history = model.fit(X_train, y_train, epochs=500, batch_size=2048)

Epoch 1/500


4/4 [==============================] - 1s 46ms/step - loss: 1.8188
Epoch 2/500
4/4 [==============================] - 0s 50ms/step - loss: 1.4728
Epoch 3/500
4/4 [==============================] - 0s 50ms/step - loss: 1.3850
Epoch 4/500
4/4 [==============================] - 0s 45ms/step - loss: 1.3141
Epoch 5/500
4/4 [==============================] - 0s 47ms/step - loss: 1.2663
Epoch 6/500
4/4 [==============================] - 0s 80ms/step - loss: 1.2311
Epoch 7/500
4/4 [==============================] - 0s 49ms/step - loss: 1.2046
Epoch 8/500
4/4 [==============================] - 0s 80ms/step - loss: 1.1853
Epoch 9/500
4/4 [==============================] - 0s 50ms/step - loss: 1.1749
Epoch 10/500
4/4 [==============================] - 0s 48ms/step - loss: 1.1630
Epoch 11/500
4/4 [==============================] - 0s 47ms/step - loss: 1.1557
Epoch 12/500
4/4 [==============================] - 0s 50ms/step - loss: 1.1484
Epoch 13/500
4/4 [==============================] - 0s 49ms/s

In [468]:
# plot history with plotly
fig = px.line(
    history.history,
    labels={'index': 'epoch', 'value': 'loss'},
    title='Loss over epochs'
)
fig.show()

In [469]:
# evaluate model
model.evaluate(X_test, y_test)

# make predictions
y_pred = model.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns=df_y.columns)
y_pred.index = y_test.index
y_pred

59/59 [==============================] - 0s 959us/step


,CO(mg/m^3)_lag_1,CO(mg/m^3)_lag_2,CO(mg/m^3)_lag_3,CO(mg/m^3)_lag_4,CO(mg/m^3)_lag_5,CO(mg/m^3)_lag_6
1518,1.612189,1.383861,1.191736,0.868238,0.798736,1.120595
5987,0.617057,2.052753,4.160100,3.733737,2.427637,2.114819
7111,1.412539,1.156363,0.978853,0.834937,0.863630,1.337237
966,1.270139,1.234046,1.233479,1.237700,1.234560,1.238146
2777,2.042418,1.794403,1.617498,1.662734,1.964189,2.313808
...,...,...,...,...,...,...
617,1.929957,2.198735,2.376197,2.873509,2.727328,3.545862
6885,3.027662,3.733970,4.658151,4.997126,4.809875,3.931974
3767,0.800618,1.010124,1.342170,1.375713,1.305691,1.695930
263,2.900113,3.079849,2.910530,2.234977,1.957982,1.912797


In [470]:
# calculate mean absolute error mean squared error and mean absolute percentage error for each column
mae_mean = 0
for col in y_test.columns:
    mae = np.mean(np.abs(y_test[col] - y_pred[col]))
    mae_mean += mae
    print(f'{col}: MAE: {mae}')
print(f'Mean MAE: {mae_mean/6}')

CO(mg/m^3)_lag_1: MAE: 0.3926831388089995
CO(mg/m^3)_lag_2: MAE: 0.4968563914970162
CO(mg/m^3)_lag_3: MAE: 0.5501705725838607
CO(mg/m^3)_lag_4: MAE: 0.5715380364944085
CO(mg/m^3)_lag_5: MAE: 0.628628765641844
CO(mg/m^3)_lag_6: MAE: 0.6424215457554477
Mean MAE: 0.5470497417969294


In [471]:
# add prefix to column names
y_pred.columns = 'pred_' + y_pred.columns
y_test.columns = 'true_' + y_test.columns
# compare predictions with actual values
y_compare = pd.concat([y_test, y_pred], axis=1)
y_compare

,true_CO(mg/m^3)_lag_1,true_CO(mg/m^3)_lag_2,true_CO(mg/m^3)_lag_3,true_CO(mg/m^3)_lag_4,true_CO(mg/m^3)_lag_5,true_CO(mg/m^3)_lag_6,pred_CO(mg/m^3)_lag_1,pred_CO(mg/m^3)_lag_2,pred_CO(mg/m^3)_lag_3,pred_CO(mg/m^3)_lag_4,pred_CO(mg/m^3)_lag_5,pred_CO(mg/m^3)_lag_6
1518,2.10,1.30,1.00,1.10,1.20,1.50,1.612189,1.383861,1.191736,0.868238,0.798736,1.120595
5987,0.50,0.90,3.50,5.20,4.30,3.50,0.617057,2.052753,4.160100,3.733737,2.427637,2.114819
7111,1.30,1.30,1.30,1.10,1.00,1.00,1.412539,1.156363,0.978853,0.834937,0.863630,1.337237
966,1.25,1.25,1.25,1.25,1.25,1.25,1.270139,1.234046,1.233479,1.237700,1.234560,1.238146
2777,2.20,2.10,1.80,2.00,1.80,3.00,2.042418,1.794403,1.617498,1.662734,1.964189,2.313808
...,...,...,...,...,...,...,...,...,...,...,...,...
617,2.60,2.90,2.50,1.90,1.90,2.70,1.929957,2.198735,2.376197,2.873509,2.727328,3.545862
6885,4.30,6.60,8.50,8.70,9.10,7.20,3.027662,3.733970,4.658151,4.997126,4.809875,3.931974
3767,0.80,1.00,1.10,1.10,1.60,0.90,0.800618,1.010124,1.342170,1.375713,1.305691,1.695930
263,3.80,4.40,4.50,3.30,2.00,1.10,2.900113,3.079849,2.910530,2.234977,1.957982,1.912797


In [472]:
# plot predictions
fig = px.line(y_compare.iloc[:100][["true_CO(mg/m^3)_lag_1", "pred_CO(mg/m^3)_lag_1"]], title='Predictions vs true values')
fig.show()

In [473]:
# plot predictions
fig = px.line(y_compare.iloc[:100][["true_CO(mg/m^3)_lag_2", "pred_CO(mg/m^3)_lag_2"]], title='Predictions vs true values')
fig.show()

In [474]:
# plot predictions
fig = px.line(y_compare.iloc[:100][["true_CO(mg/m^3)_lag_6", "pred_CO(mg/m^3)_lag_6"]], title='Predictions vs true values')
fig.show()

In [475]:
# fit the model on the test set and save it as pkl
model.fit(X_test, y_test, epochs=500, batch_size=2048)

Epoch 1/500
1/1 [==============================] - 0s 52ms/step - loss: 0.5615
Epoch 2/500
1/1 [==============================] - 0s 51ms/step - loss: 0.5522
Epoch 3/500
1/1 [==============================] - 0s 63ms/step - loss: 0.5381
Epoch 4/500
1/1 [==============================] - 0s 54ms/step - loss: 0.5243
Epoch 5/500
1/1 [==============================] - 0s 54ms/step - loss: 0.5135
Epoch 6/500
1/1 [==============================] - 0s 53ms/step - loss: 0.4996
Epoch 7/500
1/1 [==============================] - 0s 52ms/step - loss: 0.4876
Epoch 8/500
1/1 [==============================] - 0s 57ms/step - loss: 0.4761
Epoch 9/500
1/1 [==============================] - 0s 52ms/step - loss: 0.4672
Epoch 10/500
1/1 [==============================] - 0s 62ms/step - loss: 0.4588
Epoch 11/500
1/1 [==============================] - 0s 56ms/step - loss: 0.4501
Epoch 12/500
1/1 [==============================] - 0s 55ms/step - loss: 0.4388
Epoch 13/500
1/1 [==============================]

/home/davide/Desktop/Air_Control/Simulate_air_quality_control/env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [ ]:
model.save("predict_model.keras")